In [1]:
import os
import json
import requests
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
model = "gemini-pro"
prompt = "北京下大雪了，以李白的风格写一首诗：\n"
url = f"https://googleapis.fmr.wiki/v1beta/models/{model}:generateContent?key={os.getenv('GOOGLE_API_KEY')}"
headers = {"Content-Type": "application/json"}
data = {
    "contents": [{
        "parts":[{
            "text": prompt
        }]
    }]
}

In [3]:
ret = requests.post(url, json.dumps(data), headers=headers)

In [4]:
text = ret.json()['candidates'][0]['content']['parts'][0]['text']

In [5]:
print(text)

北风怒号雪飞扬，玉宇琼楼下界来。
万里江山披银装，九州大地一片白。

银装素裹分外妖，琼枝玉叶舞翩跹。
恰似天仙下凡尘，人间仙境入画来。

雪花飞舞如柳絮，柳絮轻扬似雪花。
忽散忽聚难捉摸，缥缈虚无如梦境。

雪压枝头花不惧，迎风傲雪绽芬芳。
梅花独傲凌寒开，风骨铮铮傲然在。

银色世界美如画，白雪皑皑壮如歌。
赞叹伟大造物主，奇观异景醉人心。

同享冰雪童话趣，共赏北国冬日美。
寒冬时节暖意浓，雪中洋溢欢声笑。

诗成雪落观梅时，一碗清酒醉己身。
人生得意须尽欢，且行且珍惜眼前。


In [6]:
model = "gemini-pro-vision"
prompt = '''As a genius expert, your task is to understand the content and provide the parsed objects in json that match the following json_schema:

{'result': {'default': [], 'items': {'$ref': '#/$defs/ResultDetail'}, 'title': 'Result', 'type': 'array'}, 'description': {'title': 'Description', 'type': 'string'}}
                
Here are some more definitions to adhere too:
{'Category': {'properties': {'primary': {'title': 'Primary', 'type': 'string'}, 'secondary': {'title': 'Secondary', 'type': 'string'}}, 'required': ['primary', 'secondary'], 'title': 'Category', 'type': 'object'}, 'ResultDetail': {'properties': {'exist': {'title': 'Exist', 'type': 'string'}, 'category': {'anyOf': [{'$ref': '#/$defs/Category'}, {'type': 'null'}], 'default': None}, 'freshness': {'title': 'Freshness', 'type': 'string'}, 'description': {'title': 'Description', 'type': 'string'}}, 'required': ['exist', 'freshness', 'description'], 'title': 'ResultDetail', 'type': 'object'}}

任务简介：
- 识别每张图片中的主要物品，判断其分类并评估新旧程度。

分类标准：
- 服装：大衣、皮衣、半袖、裤子、裙子、内衣裤、秋衣类、毛衣类、工装、校服。
- 家电：空气净化器、厨房家电、家居家电。
- 乐器：电子乐器、琴类。
- 玩具：小件（手办）、大件、毛绒玩具。
- 图书：儿童绘本、课外书、小说、套装书籍、课本。
- 手机：智能手机、功能机。
- 笔记本电脑：品牌电脑、非品牌电脑。

对于没有给出分类的物品，需要你自行判断一级分类（primary）和二级分类（secondary），此时exist设置为"否"。
'''
url = f"https://googleapis.fmr.wiki/v1beta/models/{model}:generateContent?key={os.getenv('GOOGLE_API_KEY')}"
headers = {"Content-Type": "application/json"}

In [7]:
import base64

def image_data(image):
    return base64.b64encode(requests.get(image).content).decode()

def describe_multiple(images, prompt=prompt):
    data = {
        "contents": [{
            "parts":[{"text": prompt}] +
                [{"inline_data": {"mime_type":"image/jpeg","data": image_data(image)}} for image in images]
        }]
    }
    resp = requests.post(url, json.dumps(data), headers=headers)
    return resp

In [8]:
urls = [f"https://milo-test.oss-cn-zhangjiakou.aliyuncs.com/hdd/batch1/image{i+1:03}.png" for i in range(241)]

In [9]:
%%time
ret = describe_multiple(urls[:10])

CPU times: user 461 ms, sys: 51.3 ms, total: 512 ms
Wall time: 33.5 s


In [10]:
result = ret.json()['candidates'][0]['content']['parts'][0]['text']

In [11]:
from typing import List
from pydantic import BaseModel

class Category(BaseModel):
    primary: str
    secondary: str

class ResultDetail(BaseModel):
    exist: str
    category: Category | None = None
    freshness: str
    description: str

class ResultList(BaseModel):
    result: List[ResultDetail] = []
    description: str = None

In [12]:
import json

In [13]:
data = json.loads(result.strip().lstrip("```json").rstrip("```"))

In [14]:
ret = ResultList(**data)

In [15]:
from IPython.display import Image, display, HTML

In [16]:
table = "<table><tr><th>Text</th><th>Image</th></tr>"
for i in range(10):
    table += f"<tr><td><img src='{urls[i]}' width='100'></td><td>{ret.result[i]}</td></tr>"  

In [17]:
display(HTML(table))

Text,Image
,"exist='是' category=Category(primary='服装', secondary='童鞋') freshness='新' description='白色鞋子，有魔术贴'"
,"exist='是' category=Category(primary='家居家电', secondary='其他') freshness='新' description='蓝色的鼻喷雾剂'"
,"exist='是' category=Category(primary='服装', secondary='休闲鞋') freshness='旧' description='灰色帆布鞋，有鞋带'"
,"exist='是' category=Category(primary='图书', secondary='杂志') freshness='旧' description='《故事会》2017年9月刊'"
,"exist='是' category=Category(primary='图书', secondary='课本') freshness='新' description='习近平新时代中国特色社会主义思想学习纲要'"
,"exist='是' category=Category(primary='家居家电', secondary='其他') freshness='新' description='木质搓衣板'"
,"exist='是' category=Category(primary='服装', secondary='凉拖') freshness='旧' description='绿色人字拖，有卡通图案'"
,"exist='是' category=Category(primary='服装', secondary='婴儿背带') freshness='旧' description='红色婴儿背带'"
,"exist='是' category=Category(primary='服装', secondary='T恤') freshness='新' description='蓝色短袖T恤，有领标'"
,"exist='是' category=Category(primary='厨房家电', secondary='咖啡机') freshness='新' description='伊莱克斯咖啡机'"
